In [1]:
import argparse

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import RandomSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = "cuda:0"  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)

load gdsc2


In [7]:
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = RandomSampler(res, train_index, test_index, null_mask)
    model = nihgcn(
        adj_mat=sampler.train_data,
        cell_exprs=exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    ).to(args.device)
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)], ignore_index=True)
    predict_datas = pd.concat(
        [predict_datas, translate_result(predict_data)], ignore_index=True
    )

epoch:   0 loss:0.705132 auc:0.4852
epoch:  20 loss:0.138467 auc:0.9684
epoch:  40 loss:0.123824 auc:0.9741
epoch:  60 loss:0.111770 auc:0.9783
epoch:  80 loss:0.103934 auc:0.9807
epoch: 100 loss:0.098278 auc:0.9819
epoch: 120 loss:0.092646 auc:0.9830
epoch: 140 loss:0.087156 auc:0.9837
epoch: 160 loss:0.082540 auc:0.9839
epoch: 180 loss:0.080363 auc:0.9832
epoch: 200 loss:0.076553 auc:0.9840
epoch: 220 loss:0.074182 auc:0.9839
epoch: 240 loss:0.072330 auc:0.9840
epoch: 260 loss:0.072442 auc:0.9835
epoch: 280 loss:0.070156 auc:0.9841
epoch: 300 loss:0.069760 auc:0.9836
epoch: 320 loss:0.068500 auc:0.9842
epoch: 340 loss:0.067958 auc:0.9842
epoch: 360 loss:0.068261 auc:0.9842
epoch: 380 loss:0.067340 auc:0.9840
epoch: 400 loss:0.068031 auc:0.9841
epoch: 420 loss:0.066811 auc:0.9838
epoch: 440 loss:0.066503 auc:0.9840
epoch: 460 loss:0.066475 auc:0.9839
epoch: 480 loss:0.066303 auc:0.9841
epoch: 500 loss:0.067673 auc:0.9826
epoch: 520 loss:0.065775 auc:0.9839
epoch: 540 loss:0.066122 auc

In [8]:
true_datas.to_csv("true_gdsc2.csv")
predict_datas.to_csv("pred_gdsc2.csv")